In [1]:
#importing open-source libraries required
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [4]:
stocks = pd.read_csv('sp500_companies.csv')
stocks

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,248.05,3749474205696,1.346610e+11,0.061,Cupertino,CA,United States,164000.0,"Apple Inc. designs, manufactures, and markets ...",0.068113
1,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software - Infrastructure,437.39,3251942195200,1.365520e+11,0.160,Redmond,WA,United States,228000.0,Microsoft Corporation develops and supports so...,0.059075
2,NMS,NVDA,NVIDIA Corporation,NVIDIA Corporation,Technology,Semiconductors,128.91,3157005959168,6.118400e+10,1.224,Santa Clara,CA,United States,29600.0,NVIDIA Corporation provides graphics and compu...,0.057350
3,NMS,AMZN,"Amazon.com, Inc.","Amazon.com, Inc.",Consumer Cyclical,Internet Retail,220.52,2318768013312,1.115830e+11,0.110,Seattle,WA,United States,1551000.0,"Amazon.com, Inc. engages in the retail sale of...",0.042123
4,NMS,GOOG,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,190.15,2316635471872,1.234700e+11,0.151,Mountain View,CA,United States,181269.0,Alphabet Inc. offers various products and plat...,0.042084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,NMS,CZR,"Caesars Entertainment, Inc.","Caesars Entertainment, Inc.",Consumer Cyclical,Resorts & Casinos,33.88,7198822400,3.668000e+09,-0.040,Reno,NV,United States,51000.0,"Caesars Entertainment, Inc. operates as a gami...",0.000131
499,NYQ,BWA,BorgWarner Inc.,BorgWarner Inc.,Consumer Cyclical,Auto Parts,31.95,6987465216,1.882000e+09,-0.048,Auburn Hills,MI,United States,39900.0,"BorgWarner Inc., together with its subsidiarie...",0.000127
500,NMS,QRVO,"Qorvo, Inc.","Qorvo, Inc.",Technology,Semiconductors,68.50,6475079168,6.731300e+08,-0.052,Greensboro,NC,United States,8700.0,"Qorvo, Inc. engages in development and commerc...",0.000118
501,NYQ,FMC,FMC Corporation,FMC Corporation,Basic Materials,Agricultural Inputs,49.11,6130696192,7.033000e+08,0.085,Philadelphia,PA,United States,5800.0,"FMC Corporation, an agricultural sciences comp...",0.000111


In [5]:
from datetime import datetime, timedelta

In [6]:
import refinitiv.data as rd


In [7]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x2773d84f5f0 {name='workspace'}>

In [8]:
sp500_constituents = rd.get_data(
    ["0#.SPX"],
    ["TR.RIC"]
)

#> this retrieves the list of all S&P500 company RIC (reuters instrument code) for querying of stock data
#> not to forget this initiates a session with refinitiv API

sp500_ric = sp500_constituents['RIC'].tolist()

In [9]:
from datetime import datetime, timedelta

startingDate = datetime(2023, 10, 1)
endingDate = datetime(2024, 10, 1)

df = rd.get_history(
    universe=sp500_ric,
    fields=["TR.PriceClose"],
    interval="1D",
    start=startingDate,
    end=endingDate
)

#> the stock price data ('TR.PriceClose') is pulled for all S&P 500 stocks over a 1-year period (from October 1, 2023, to October 1, 2024) at a daily basis. The data is stored in 'df'.

C:\Users\Ubayd Knight\PyCharmMiscProject\.venv\Lib\site-packages\refinitiv\data\_tools\_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [10]:
import time

batch_size = 50
all_data = []

for i in range(0, len(sp500_ric), batch_size):
    batch = sp500_ric[i:i+batch_size]
    batch_data = rd.get_history(
        universe=batch,
        fields=["TR.PriceClose"],
        interval="1D",
        start=startingDate,
        end=endingDate
    )
    all_data.append(batch_data)
    time.sleep(1)

C:\Users\Ubayd Knight\PyCharmMiscProject\.venv\Lib\site-packages\refinitiv\data\_tools\_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [12]:
df.columns

Index(['POOL.OQ', 'CHRW.OQ', 'AJG.N', 'CNP.N', 'AMCR.N', 'WM.N', 'BA.N',
       'FOX.OQ', 'WY.N', 'MCD.N',
       ...
       'EQT.N', 'NRG.N', 'MKTX.OQ', 'LMT.N', 'DLR.N', 'PSX.N', 'SCHW.N',
       'SNPS.OQ', 'J.N', 'AVY.N'],
      dtype='object', name='Price Close', length=503)

In [20]:
df.to_excel("momentumData.xlsx", sheet_name="data", index=True)
print("Done.")

Done.


In [14]:
import pandas as pd

df = df.reset_index()
df_long = pd.melt(df, id_vars=['Date'], var_name='Instrument', value_name='Close')
df_long = df_long.set_index(['Date', 'Instrument'])

momentum_period = 12
df_long['Momentum'] = df_long['Close'].groupby(level=1).pct_change(periods=momentum_period)
df_long['Momentum_Rank'] = df_long['Momentum'].groupby(level=0).rank(ascending=False, method='first')
df_export = df_long.reset_index()

df_export.to_excel('momentum_data.xlsx', sheet_name='Momentum Analysis', index=False)
print("Data saved to momentum_data.xlsx")

C:\Users\Ubayd Knight\AppData\Local\Temp\ipykernel_57736\3978053518.py:8:FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.


Data saved to momentum_data.xlsx


In [15]:
df

Price Close,Date,POOL.OQ,CHRW.OQ,AJG.N,CNP.N,AMCR.N,WM.N,BA.N,FOX.OQ,WY.N,...,EQT.N,NRG.N,MKTX.OQ,LMT.N,DLR.N,PSX.N,SCHW.N,SNPS.OQ,J.N,AVY.N
0,2023-10-02,349.0,85.65,224.91,25.75,9.06,151.21,187.83,28.57,29.852915,...,39.09,37.57,209.95,407.82,118.22,119.02,53.55,463.82,112.184016,181.52
1,2023-10-03,338.85,85.71,225.22,26.35,8.93,152.56,188.92,28.03,29.374949,...,39.62,37.85,204.76,403.83,115.15,115.63,51.3,450.72,111.173726,179.5
2,2023-10-04,338.55,85.51,229.26,26.54,9.05,153.96,186.73,28.05,29.783212,...,39.65,37.46,213.28,401.33,117.56,110.49,51.41,461.32,112.693336,183.32
3,2023-10-05,333.78,84.35,229.56,26.62,8.93,154.72,186.29,27.98,29.892745,...,41.09,38.69,225.61,397.35,117.7,109.48,51.06,459.75,112.209065,182.79
4,2023-10-06,336.12,84.86,232.01,26.87,8.85,155.44,187.38,28.13,29.693593,...,42.79,38.97,238.73,400.73,117.37,109.19,51.52,473.26,114.889258,183.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2024-09-25,366.07,108.02,280.75,28.62,11.08,207.69,152.22,38.44,32.97,...,35.16,89.24,254.38,577.94,162.9,129.85,64.04,510.36,128.64925,218.82
248,2024-09-26,370.33,107.54,277.99,28.94,11.35,205.34,154.58,38.67,33.16,...,35.51,90.58,254.19,577.4,162.06,130.68,63.84,515.4,129.091774,221.76
249,2024-09-27,372.54,108.34,279.45,29.1,11.35,205.51,156.32,38.84,33.75,...,36.53,91.22,254.84,582.36,161.16,131.05,64.42,513.31,129.859929,222.34
250,2024-09-30,376.8,110.37,281.37,29.42,11.33,207.6,152.04,38.8,33.86,...,36.64,91.1,256.2,584.56,161.83,131.45,64.81,506.39,130.9,220.76


In [16]:
momentum_threshold = 25
df_filtered = df_long[df_long['Momentum_Rank'] <= momentum_threshold]

df_filtered = df_filtered.reset_index()

df_filtered.to_excel('momentum_data_filtered.xlsx', sheet_name='Filtered Momentum Analysis', index=False)

print("Filtered data saved to momentum_data_filtered.xlsx")

Filtered data saved to momentum_data_filtered.xlsx


In [ ]:
import pandas as pd

def createPortfolio(df, portfolio_size, shares_per_stock, weight_by_momentum):
    df = df.drop_duplicates(subset='Instrument', keep='first')
    df_sorted = df.sort_values(by='Momentum_Rank')

    if len(df_sorted) < portfolio_size:
        print("Warning: Not enough unique stocks for the desired portfolio size.")
        portfolio_size = len(df_sorted)

    if weight_by_momentum:
        df_sorted['Suggested No. of Shares'] = df_sorted['Momentum'].rank(method='dense', ascending=True).map(
            lambda x: shares_per_stock * (x / df_sorted['Momentum'].rank(method='dense', ascending=False).max()))
        df_sorted['Suggested No. of Shares'] = df_sorted['Suggested No. of Shares'].astype(int)
    else:
        df_sorted['Suggested No. of Shares'] = shares_per_stock

    portfolio = df_sorted.head(portfolio_size)[['Instrument', 'Momentum_Rank', 'Suggested No. of Shares']]
    portfolio.rename(columns={'Instrument': 'Stocks', 'Momentum_Rank': 'Momentum Rank'}, inplace=True)

    portfolio = portfolio.reset_index(drop=True)

    return portfolio

def setupPortfolio():
    df_filtered = pd.read_excel("momentum_data_filtered.xlsx", engine="openpyxl")

    portfolioSize = int(input("How many stocks would you like in your portfolio? "))
    sharesPerStock = int(input("How many shares would you like per stock? "))
    weightMomentum = input("Would you like to weigh the shares by momentum? (yes/no): ").lower() == 'yes'

    portfolio = createPortfolio(df_filtered, portfolio_size=portfolioSize, shares_per_stock=sharesPerStock,
                                weight_by_momentum=weightMomentum)

    file_name = input("Enter a file name to save the portfolio (e.g., 'portfolio.xlsx'): ")
    portfolio.to_excel(file_name, index=False)

    print("\nYour Portfolio has been created and saved successfully!")
    print(f"File saved as {file_name}. Check your working directory.")

    print("\nYour Portfolio has been created!")
    print(portfolio)

if __name__ == "__main__":
    setupPortfolio()


Your Portfolio has been created and saved successfully!
File saved as utartizan.xlsx. Check your working directory.

Your Portfolio has been created!
    Stocks  Momentum Rank  Suggested No. of Shares
0     GL.N              1                     1500
1  ANSS.OQ              1                     1496
2    ALL.N              1                     1199
3   CTVA.N              2                     1391
4  PLTR.OQ              2                     1177
5   DECK.N              2                     1068
6    PGR.N              2                     1181
7   JNPR.N              2                     1492
8    UNH.N              3                      311
9  ULTA.OQ              3                     1384
